# Welcome to nxneo4j!
#### nxneo4j is a library that enables you to use networkX type of commands to interact with Neo4j. 

### _Latest version is 0.0.3_
If not already installed, install the latest version like this:

In [ ]:
! pip uninstall -y networkx-neo4j #remove the old installation

In [ ]:
! pip install git+https://github.com/ybaktir/networkx-neo4j #install the latest one

In [1]:
import datetime, time
print ('Last run on: ' + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") + ' ' + repr(time.tzname))

Last run on: 2020-08-31 19:54:34 ('CST', 'CDT')


## Connect to Neo4j

In [2]:
from neo4j import GraphDatabase

In [4]:
driver = GraphDatabase.driver(uri="bolt://localhost:11007",auth=("neo4j","your_password"))
                              #OR "bolt://localhost:7673"
                              #OR the cloud url

In [5]:
import nxneo4j as nx

In [6]:
G = nx.Graph(driver)

In [7]:
G.delete_all()  #This will delete all the data, be careful
                #Just making sure that the results are reprodusible.

## Add Nodes

In [8]:
#Add a node
G.add_node("Yusuf")

In [9]:
#Add node with features
G.add_node("Nurgul",gender='F')

In [10]:
#Add multiple properties at once
G.add_node("Betul",age=4,gender='F')

In [11]:
#Check nodes
for node in G.nodes():   #Unlike networkX, nxneo4j returns a generator
    print(node)

Yusuf
Nurgul
Betul


In [12]:
#Or simply
list(G.nodes())

['Yusuf', 'Nurgul', 'Betul']

In [13]:
#Get the data associated with each node
list(G.nodes(data=True))

[('Yusuf', {}),
 ('Nurgul', {'gender': 'F'}),
 ('Betul', {'gender': 'F', 'age': 4})]

In [14]:
#number of nodes
len(G)

3

In [15]:
#Display
nx.draw(G) #It is interactive, drag the nodes!

In [16]:
#Check a particular node feature
G.nodes['Betul']

{'gender': 'F', 'age': 4}

In [17]:
#You can be more specific
G.nodes['Betul']['age']

4

In [18]:
G.add_nodes_from([1,2,3,4])

In [19]:
list(G.nodes())

['Yusuf', 'Nurgul', 'Betul', 1, 2, 3, 4]

## Add Edges

In [20]:
#Add one edge
G.add_edge('Yusuf','Max')

In [21]:
nx.draw(G) #default relationship label is "CONNECTED"

In [22]:
#You can change the default connection label like the following
G.relationship_type = 'LOVES'

In [23]:
G.add_edge('Yusuf','Nurgul')
G.add_edge('Nurgul','Yusuf')

In [24]:
nx.draw(G)

In [25]:
#You can add properties as well
G.add_edge('Betul','Nurgul',how_much='More than Dad')

In [26]:
#display the values
list(G.edges(data=True))

[('Yusuf', 'Nurgul', {}),
 ('Nurgul', 'Yusuf', {}),
 ('Betul', 'Nurgul', {'how_much': 'More than Dad'})]

In [27]:
G.relationship_type = 'CONNECTED'

In [28]:
G.add_edges_from([(1,2),(3,4)])

In [29]:
nx.draw(G)

## Remove Nodes

In [30]:
G.remove_node('Yusuf')

In [31]:
list(G.nodes())

['Nurgul', 'Betul', 1, 2, 3, 4, 'Max']

## Graph Data Science

There are several builtin graph algorithms in Neo4j. nxneo4j will expand to cover all of them in the future versions. For now, the following networkX algorithms are supported: 
- pagerank
- betweenness_centrality
- closeness_centrality
- label_propagation
- connected_components
- clustering 
- triangles
- shortest_path
- shortest_weighted_path

Let's delete all data and load GOT data:

In [32]:
G.delete_all()
G.load_got()

In [33]:
#You can change the default parameters like the following:
G.identifier_property = 'name'
G.relationship_type = '*'
G.node_label = 'Character'

In [34]:
nx.draw(G) #Zoom in to see the names :)

In [34]:
len(G) #796 nodes

796

## 1. Centrality Algorithms

We’ll start with the famous PageRank algorithm. Let’s find out who the most influential characters in Game of Thrones are:

### Pagerank

We’ll start with the famous PageRank algorithm. Let’s find out who the most influential characters in Game of Thrones are:

In [ ]:
nx.pagerank(G) #RAW OUTPUT

In [36]:
# the most influential characters
response = nx.pagerank(G)
sorted_pagerank = sorted(response.items(), key=lambda x: x[1], reverse=True)
for character, score in sorted_pagerank[:10]:
    print(character, score)

Jon-Snow 17.596909502156667
Tyrion-Lannister 17.568136240123653
Jaime-Lannister 13.925499376200438
Cersei-Lannister 13.402380343770089
Daenerys-Targaryen 12.499217151004583
Stannis-Baratheon 12.15039813708843
Arya-Stark 11.69211189582387
Robb-Stark 11.277725861477968
Eddard-Stark 10.68388151188578
Catelyn-Stark 10.619218634539562


### Betweenness centrality

We can also run betweenness centrality over the dataset. This algorithm will tell us which nodes are the most 'pivotal' i.e. how many of the shortest paths between pairs of characters must pass through them

In [ ]:
# Betweenness centrality
nx.betweenness_centrality(G) #RAW OUTPUT

In [38]:
# RANKED OUTPUT
response = nx.betweenness_centrality(G)

sorted_bw = sorted(response.items(), key=lambda x: x[1], reverse=True)
for character, score in sorted_bw[:10]:
    print(character, score)

Jon-Snow 65395.26787165435
Tyrion-Lannister 50202.17398521847
Daenerys-Targaryen 39636.77718662114
Stannis-Baratheon 35984.21182863314
Theon-Greyjoy 35436.85268519103
Jaime-Lannister 32122.976615424588
Robert-Baratheon 31391.065251945023
Arya-Stark 29342.15853062157
Cersei-Lannister 28274.915426635584
Eddard-Stark 26470.250249098248


### Closeness centrality

Closeness centrality tells us on average how many hops away each character is from every other character.

In [ ]:
# Closeness centrality
nx.closeness_centrality(G) #RAW OUTPUT

In [40]:
# RANKED
response = nx.closeness_centrality(G)

sorted_cc = sorted(response.items(), key=lambda x: x[1], reverse=True)
for character, score in sorted_cc[:10]:
    print(character, score)

Tyrion-Lannister 0.4763331336129419
Robert-Baratheon 0.4592720970537262
Eddard-Stark 0.455848623853211
Cersei-Lannister 0.45454545454545453
Jaime-Lannister 0.4519613416714042
Jon-Snow 0.44537815126050423
Stannis-Baratheon 0.4446308724832215
Robb-Stark 0.4441340782122905
Joffrey-Baratheon 0.4339519650655022
Catelyn-Stark 0.4334787350054526


## 2. Community Detection Algoritms

### Label Propagation
We can also partition the characters into communities using the label propagation algorithm

In [41]:
# Label propagation
nx.label_propagation_communities(G) #RAW OUPUT is a generator

<generator object label_propagation_communities at 0x1071ba0d0>

In [42]:
communities = nx.label_propagation_communities(G)
sorted_communities = sorted(communities, key=lambda x: len(x), reverse=True)
for community in sorted_communities[:10]:
    print(list(community)[:10])

['Leo-Lefford', 'Ravella-Swann', 'Raynald-Westerling', 'Harwood-Stout', 'Guncer-Sunglass', 'Gawen-Westerling', 'Shagwell', 'Maron-Greyjoy', 'Sarella-Sand', 'Harl']
['Xhondo', 'Orell', 'Wynton-Stout', 'Dalla', 'Tormund', 'Quhuru-Mo', 'Owen', 'Val', 'Pate-(novice)', 'Othor']
['Qezza', 'Draqaz', 'Reznak-mo-Reznak', 'Hugh-Hungerford', 'Rakharo', 'Fogo', 'Ogo', 'Meris', 'Kraznys-mo-Nakloz', 'Kedry']


Characters are in the same community as those other characters with whom they frequently interact. The idea is that characters have closer ties to those in their community than to those outside.



### Clustering
We can calculate the clustering coefficient for each character. A clustering coefficient of '1' means that all characters that interact with that character also interact with each other:

In [ ]:
# Clustering
nx.clustering(G) #RAW OUTPUT

In [44]:
response = nx.clustering(G)

biggest_coefficient = sorted(response.items(), key=lambda x: x[1], reverse=True)
for character in biggest_coefficient[:10]:
    print(list(character)[:10])

['Steffon-Baratheon', 4.0]
['Oswell-Kettleblack', 4.0]
['Wylis-Manderly', 4.0]
['Beth-Cassel', 3.0]
['Big-Boil', 3.0]
['Dirk', 3.0]
['Jon-Umber-(Smalljon)', 3.0]
['Orell', 3.0]
['Oznak-zo-Pahl', 3.0]
['Mag-Mar-Tun-Doh-Weg', 3.0]


In [ ]:
list(nx.connected_components(G))

In [46]:
nx.number_connected_components(G)

1

In [ ]:
nx.triangles(G) #RAW OUTPUT

## 3. Path Finding Algorithms

Let's find the distance between two characters

In [48]:
# Shortest path
nx.shortest_path(G, source="Tyrion-Lannister", target="Hodor")

['Tyrion-Lannister', 'Luwin', 'Hodor']

In [49]:
# Shortest weighted path
nx.shortest_weighted_path(G, source="Tyrion-Lannister", target="Hodor",weight='weight')

['Tyrion-Lannister', 'Theon-Greyjoy', 'Wyman-Manderly', 'Hodor']